In [8]:
#Import useful functions
import warnings
warnings.filterwarnings("ignore")
import itertools
import xlrd
import xlwt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pylab as plt
import numpy as np
import xlsxwriter
from xlrd import open_workbook
from datetime import datetime
plt.style.use('fivethirtyeight')

In [9]:
#Designate filename of original tick data excel file
filenameorig = '/Users/JoyThompson/Desktop/Tick Data/JoyTickData.xlsx'
#Read the excel file and write to a pandas dataframe
tick = pd.read_excel(filenameorig, sheetname='Breakdown')
#Set the index of the tick data dataframe to the index to parse through all data values
#Designate the filename of the excel file to write the new dataframe to
filenameorig2= '/Users/JoyThompson/Desktop/Tick Data/ckemployees.xlsx'
#Initialize excel writer to open and write to the above designated file
ckem = pd.read_excel(filenameorig2, sheetname='Sheet1')
ckem=ckem.set_index('Employee')
filenamefinal= '/Users/JoyThompson/Desktop/Tick Data/Tick Data Summary.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')

In [10]:
tickdata=tick
tickdata['Department']=''
for ind in tickdata.index:
    em=tickdata.loc[ind, 'User']
    dep=ckem.loc[em, 'Department']
    tickdata.loc[ind, 'Department']=dep

In [11]:
tickdatag=tickdata.groupby('Department').sum()
tickdatag=tickdatag.drop(columns=['Project Total', 'Task Total'])
tickdatag=tickdatag.rename(columns={'User Total':'Department Total'})

In [12]:
tickem=tickdata.groupby('User').sum()
tickem=tickem.drop(columns=['Project Total', 'Task Total'])
for ind in tickem.index:
  
    dep=ckem.loc[ind, 'Department']
    tickem.loc[ind, 'Department']=dep

tickem=tickem.sort_values('Department')
tickem=tickem.reset_index()

In [13]:
tickem['Department Total']=''
deplist=[]
for ind in tickem.index:
    dep=tickem.loc[ind, 'Department']
    deptot=tickdatag.loc[dep, 'Department Total']
    if dep in deplist:
        tickem.loc[ind, 'Department']=''
        tickem.loc[ind, 'Department Total']=''
    else:
        deplist.append(dep)
        tickem.loc[ind, 'Department Total']=deptot

In [7]:
tick.to_excel(writer, sheet_name='Tick Data Complete')
tickdatag.to_excel(writer, sheet_name='Tick Data Summary')
tickem.to_excel(writer, sheet_name='Tick Employee Summary')
writer.save()